In [ ]:
# union the last few years worth of games together then perform feauture engineering

In [1]:
# imports
import pandas as pd


In [3]:
# get games data for last few years
games_0 = pd.read_excel('data/nhl_game_results_2023.xlsx', header=0)
games_1 = pd.read_excel('data/nhl_game_results_2024.xlsx', header=0)
games_2 = pd.read_excel('data/nhl_game_results_2025.xlsx', header=0)

# union together
all_games = pd.concat([games_0, games_1, games_2], ignore_index=True)

# reset index
all_games.reset_index(drop=True, inplace=True)

# inspect
all_games.info()
all_games.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Game Link      3349 non-null   object        
 1   Extra Time     3349 non-null   object        
 2   Home Team      3349 non-null   object        
 3   Away Team      3349 non-null   object        
 4   Home Score     3349 non-null   int64         
 5   Away Score     3349 non-null   int64         
 6   P1 Home Score  3349 non-null   int64         
 7   P1 Away Score  3349 non-null   int64         
 8   P2 Home Score  3349 non-null   int64         
 9   P2 Away Score  3349 non-null   int64         
 10  P3 Home Score  3349 non-null   int64         
 11  P3 Away Score  3349 non-null   int64         
 12  P4 Home Score  3349 non-null   int64         
 13  P4 Away Score  3349 non-null   int64         
 14  P5 Home Score  3349 non-null   int64         
 15  P5 Away Score  3349 n

,Game Link,Extra Time,Home Team,Away Team,Home Score,Away Score,P1 Home Score,P1 Away Score,P2 Home Score,P2 Away Score,...,P5 Home Score,P5 Away Score,P6 Home Score,P6 Away Score,P7 Home Score,P7 Away Score,Date,Time,Year,Total Score
0,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,2,1,1,1,1,0,...,0,0,0,0,0,0,2023-06-24,20:00,2023,3
1,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,5,1,1,0,2,0,...,0,0,0,0,0,0,2023-06-21,20:00,2023,6
2,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,3,5,0,1,2,3,...,0,0,0,0,0,0,2023-06-18,20:00,2023,8
3,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,8,1,3,1,3,0,...,0,0,0,0,0,0,2023-06-15,20:00,2023,9
4,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,3,4,0,1,1,3,...,0,0,0,0,0,0,2023-06-13,20:00,2023,7


In [ ]:
# inspect basic stats for numeric cols
all_games.describe()

,Home Score,Away Score,P1 Home Score,P1 Away Score,P2 Home Score,P2 Away Score,P3 Home Score,P3 Away Score,P4 Home Score,P4 Away Score,P5 Home Score,P5 Away Score,P6 Home Score,P6 Away Score,P7 Home Score,P7 Away Score,Date,Year,Total Score
count,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.0,3349.0,3349.0,3349.0,3349,3349.000000,3349.000000
mean,3.210511,2.916990,0.919678,0.830397,1.074351,0.961780,1.103613,1.026874,0.081815,0.069573,0.068976,0.063302,0.0,0.0,0.0,0.0,2024-03-05 00:39:59.283368192,2023.649149,6.127501
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2023-01-01 00:00:00,2023.000000,1.000000
25%,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2023-09-24 00:00:00,2023.000000,5.000000
50%,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2024-01-27 00:00:00,2024.000000,6.000000
75%,4.000000,4.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2024-10-12 00:00:00,2024.000000,7.000000
max,10.000000,10.000000,5.000000,6.000000,7.000000,6.000000,7.000000,6.000000,1.000000,1.000000,4.000000,4.000000,0.0,0.0,0.0,0.0,2025-11-08 00:00:00,2025.000000,17.000000
std,1.757098,1.715671,0.954677,0.903608,1.017541,0.971073,1.056788,1.033429,0.274124,0.254464,0.351251,0.344170,0.0,0.0,0.0,0.0,NaN,0.654629,2.297768


0       https://www.flashscore.com/match/hockey/edmont...
1       https://www.flashscore.com/match/hockey/edmont...
2       https://www.flashscore.com/match/hockey/edmont...
3       https://www.flashscore.com/match/hockey/edmont...
4       https://www.flashscore.com/match/hockey/edmont...
                              ...                        
3344    https://www.flashscore.com/match/hockey/minnes...
3345    https://www.flashscore.com/match/hockey/florid...
3346    https://www.flashscore.com/match/hockey/ottawa...
3347    https://www.flashscore.com/match/hockey/new-je...
3348    https://www.flashscore.com/match/hockey/dallas...
Name: Game Link, Length: 3349, dtype: object

In [ ]:
# # sample code to get odds urls from game_links
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from urllib.parse import urljoin

# driver = webdriver.Chrome()
# wait = WebDriverWait(driver, 10)

# # isolate the game links
# game_urls = all_games['Game Link']

# for url in game_urls:
#     driver.get(url)

#     # wait for the odds link to appear
#     odds_link_element = wait.until(
#         EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'odds')]"))
#     )

#     # extract href attribute
#     odds_href = odds_link_element.get_attribute("href")

#     # if the href is relative, convert to absolute
#     odds_url = urljoin(url, odds_href)

#     print(f"➡️  Found odds page: {odds_url}")

#     # navigate to odds page
#     driver.get(odds_url)

#     # then from here need to scrape the 1x2 odds data

#     # ---- do your scraping here ----
#     # example: print page title
#     print("✅ On odds page:", driver.title)

# driver.quit()
